# Notebook 4: Cifar10 Classification in Pytorch Lightning

In this notebook, we will train an image classifier for the CIFAR-10 dataset, that you already know from homework 05. Now, however, we will use the PyTorch Lightning framework which makes everything much more convenient.

In case you haven't done yet, you should definitely check out the **PyTorch Lightning Introduction** first.

## (Optional) Mount in Google Colab

In [ ]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "DL_homeworks" within your main drive folder, and put the homework there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct homework folder (e.g homework_06) is given.
# OPTIONAL: Enable GPU via Runtime --> Change runtime type --> GPU

"""
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/DL_homeworks/homework_06'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))
"""

### Set up PyTorch environment in colab

For your regular environment this should already have been installed in the previous notebooks.

In [ ]:
# Optional: install correct libraries in google colab
# !python -m pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
# !python -m pip install tensorboard==2.9.1
# !python -m pip install pytorch-lightning==2.0.0

## Imports

In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
%load_ext autoreload
%autoreload 2

### Get Device
In this homework, we'll use PyTorch Lightning to build an image classifier for the CIFAR-10 dataset. As you know from homework 05, processing a large set of images is quite computation extensive. Luckily, with PyTorch we're now able to make use of our GPU to significantly speed things up.

In case you don't have a GPU, you can run this notebook on Google Colab where you can access a GPU for free.

Of course, you can also run this notebook on your CPU only - though this is definitely not recommended.


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


## Setup TensorBoard
In the previous notebooks of this homework you've already learned how to use TensorBoard. Let's use it again to make the debugging of our network and training process more convenient. Throughout this notebook, feel free to add further logs or visualizations to your TensorBoard.

In [3]:
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs --port 6006

## Define your Network

Here instead of implementing your own model, solver and dataloader, all you have to do is defining a `LightningModule`.

We've prepared the class `exercise_code/MyPytorchModel` for you, that you'll now finalize to build an image classifier with PyTorch Lightning.

### 0. Dataset & Dataloaders
Check out the function `prepare_data` of the `CIFAR10DataModule` class that loads the dataset, using the class `torchvision.datasets.ImageFolder` (or the previous `MemoryImageFolder` dataset from Homework 02), which is very similar to the class `ImageFolderDataset` that you implemented earlier.

Implement a **transform** to pre-process the raw data (standardize it and convert it to tensors) and assign it to the variable `my_transform`. Note: On the submission server, the normalization as in the notebook 3 on data augmentation will be performed, so please make sure to use the same normalization. For convenience, we added the precomputed normalization values for you. All normalization you are defining here are tailored to your training.

In pytorch-lightning we could also include the dataset and other classes in our model, but a more reasonable way is to define it outside since it usually is used across multiple projects. If you prefer the all-in-one solution, that is great as well, but here we put it separately.

If you want to improve your performance, you can also perform extensive **data augmentation** here.

Also check out the `DataLoader` class that is used to create  `train_dataloader` and `val_dataloader` and that is very similar to your previous implementation of the DataLoader.

### 1. Define your model
Next, let's define your model. Think about a good network architecture. You're completely free here and you can come up with any network you like. (\*)

Have a look at the documentation of `torch.nn` at https://pytorch.org/docs/stable/nn.html to learn how to use use this module to build your network.

Then implement your architecture: initialize it in `__init__()` and assign it to `self.model`. This is particularly easy using `nn.Sequential()` which you only have to pass the list of your layers. 

To make your model customizable and support parameter search, don't use hardcoded hyperparameters - instead, pass them as dictionary `hparams` (here, `n_hidden` is the number of neurons in the hidden layer) when initializing `MyPytorchModel`.

Here's an easy example:

```python
        self.model = nn.Sequential(
            nn.Linear(input_size, self.hparams["n_hidden"]),
            nn.ReLU(),            
            nn.Linear(self.hparams["n_hidden"], num_classes)
        )
```

Have a look at the forward path in `forward(self, x)`, which is so easy, that you don't need to implement it yourself. As PyTorch automatically computes the gradients, that's all we need to do. No need anymore to manually calculate derivatives for the backward pass.


____
\* *The size of your final model must be less than 20 MB, which is approximately equivalent to 5 Mio. params. Note that this limit is quite lenient, you will probably need much less parameters.*

*Also, don't use convolutional layers as they've not been covered yet in the lecture and build your network with fully connected layers (```nn.Linear()```).*

### 2. Training & Validation Step
Have a look at the functions `training_step` and `validation_step` that take a batch as input and calculate the loss. 

### 3. Optimizer
Lastly, implement the function `configure_optimizers()` to define your optimizer. Here, the documentation of `torch.optim`at https://pytorch.org/docs/stable/optim.html might be helpful.

That's it. You've now finalized your `LightningModule` which has (at least) the same functionality as your previous numpy-powered image classifier.

Now let's create an instance of your `MyPytorchModel`.

In [4]:
from exercise_code.MyPytorchModel import MyPytorchModel, CIFAR10DataModule
# make sure you have downloaded the Cifar10 dataset on root: "../datasets/cifar10", if not, please check homework 02.
hparams = {}

########################################################################
# TODO: Define your hyper parameters here                             #
########################################################################

hparams = {
    'input_size': 3 * 32 * 32,      
    'hidden_size': 256,      
    'num_classes': 10,       
    'batch_size': 128,        
    'learning_rate': 0.001,  
    'loading_method': 'Image',  
    'num_workers': 2,        
}

########################################################################
#                           END OF YOUR CODE                           #
########################################################################

# Make sure you downloaded the CIFAR10 dataset already when using this cell
# since we are showcasing the pytorch inhering ImageFolderDataset that
# doesn't automatically download our data. Check homework 02

# If you want to switch to the memory dataset instead of image folder use
# hparams["loading_method"] = 'Memory'
# The default is hparams["loading_method"] = 'Image'
# You will notice that it takes way longer to initialize a MemoryDataset
# method because we have to load the data points into memory all the time.

# You might get warnings below if you use too few workers. Pytorch uses
# a more sophisticated Dataloader than the one you implemented previously.
# In particular it uses multi processing to have multiple cores work on
# individual data samples. You can enable more than workers (default=2)
# via 
# hparams['num_workers'] = 8

# Set up the data module including your implemented transforms
data_module = CIFAR10DataModule(hparams)
data_module.prepare_data()
# Initialize our model
model = MyPytorchModel(hparams)

Some tests to check whether we'll accept your model:

In [5]:
from exercise_code.Util import printModelInfo
_ = printModelInfo(model)

FYI: Your model has 1.052 params.
Model accepted!


## Fit Model with Trainer
Now it's time to train your model. 

Have a look of the documentation of `L.Trainer` at https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html to find out what arguments you can pass to define your training process. 

Then, start the training with `trainer.fit(model)`.

In [6]:
import pytorch_lightning as L
trainer = None

########################################################################
# TODO: Define your trainer.                                           #
########################################################################

trainer = L.Trainer(
    max_epochs=50,                      
    enable_checkpointing=False,     
)

########################################################################
#                           END OF YOUR CODE                           #
########################################################################

trainer.fit(model, data_module)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.1 M 
-------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.210     Total estimated model params size (MB)


Epoch 0: 100%|██████████████████████| 235/235 [00:03<00:00, 73.36it/s, v_num=73]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|███████| 235/235 [00:02<00:00, 79.57it/s, v_num=73, val_acc=0.356]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  13%|██▏              | 10/79 [00:00<00:00, 134.57it/s]


Epoch 2: 100%|███████| 235/235 [00:02<00:00, 81.06it/s, v_num=73, val_acc=0.385]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  30%|█████▏           | 24/79 [00:00<00:00, 118.17it/s]


Epoch 3: 100%|███████| 235/235 [00:02<00:00, 79.69it/s, v_num=73, val_acc=0.391]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  48%|████████▏        | 38/79 [00:00<00:00, 121.65it/s]


Epoch 4: 100%|███████| 235/235 [00:02<00:00, 78.57it/s, v_num=73, val_acc=0.425]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  66%|███████████▏     | 52/79 [00:00<00:00, 116.42it/s]


...

Epoch 47: 100%|██████| 235/235 [00:03<00:00, 78.07it/s, v_num=73, val_acc=0.516]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  91%|███████████████▍ | 72/79 [00:00<00:00, 110.49it/s]


Epoch 48: 100%|██████| 235/235 [00:02<00:00, 78.87it/s, v_num=73, val_acc=0.520]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 49: 100%|██████| 235/235 [00:03<00:00, 77.56it/s, v_num=73, val_acc=0.525]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   4%|▋                 | 3/79 [00:00<00:00, 128.22it/s]


Epoch 49: 100%|██████| 235/235 [00:03<00:00, 62.08it/s, v_num=73, val_acc=0.528]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████| 235/235 [00:03<00:00, 62.06it/s, v_num=73, val_acc=0.528]


Now that everything is working, feel free to play around with different architectures. As you've seen, it's really easy to define your model or do changes there.

You'll need at least **50%** accuracy.

<div class="alert alert-warning">
    <h3>Note: Pytorch vs Pytorch-Lightning</h3>
    <p>At this point you are actually free to submit any pytorch model. A pytorch-lightning model unifies training etc. but it inherently works like a pytorch model when you use `model.forward(...)`. For homework 06 as well as 08-10 you can submit any pytorch model you so desire. In this notebook, we gave you the shell using a pytorch-lightning model since it simplifies everything (i.e., for both you and us on both the code purity as well as the datasets (and, thus, transforms etc.) being part of the model) and you can focus on the notation for now. What you will use later is your own decision and if you prefer to submit pure pytorch code that is fine too. As an exercise for eager people: after submittag and tuning a pytorch-lightning model with the current code, try to submit a pure pytorch version of said code to see what advantages pytorch lightning brings and where you feel you have more control.
    </p>
</div>

# Save your model & Report Test Accuracy

When you've done with your **hyperparameter tuning**, have achieved **at least 50% validation accuracy** and are happy with your final model, you can save it here.

Before that, we will check again whether the number of parameters is below 5 Mi and the file size is below 20 MB.

When your final model is saved, we'll lastly report the test accuracy.

In [7]:
from exercise_code.Util import test_and_save

test_and_save(model, data_module.val_dataloader(), data_module.test_dataloader())

100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 31.13it/s]


Validation Accuracy: 52.83%
FYI: Your model has 1.052 params.
Saving model...
Checking size...
Great! Your model size is less than 20 MB and will be accepted :)
Your model has been saved and is ready to be submitted. 
NOW, let's check the test accuracy:


100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 31.35it/s]


Test Accuracy: 53.28000000000001%


You've now finished your first image classifier in PyTorch Lightning. In the next homework we will see some more complex neural networks.

To create a zip file with your submission, run the following cell:

In [8]:
from exercise_code.submit import submit_exercise

submit_exercise('../output/homework_06')

relevant folders: ['exercise_code', 'models']
notebooks files: ['2_tensorboard.ipynb', '4_Cifar10_PytorchLightning.ipynb', '1_pytorch.ipynb', '3_pytorch_lightning.ipynb']
Adding folder exercise_code
Adding folder models
Adding notebook 2_tensorboard.ipynb
Adding notebook 4_Cifar10_PytorchLightning.ipynb
Adding notebook 1_pytorch.ipynb
Adding notebook 3_pytorch_lightning.ipynb
Zipping successful! Zip is stored under: /Users/tigrangaplanyan/Downloads/DL_homeworks/output/homework_06.zip


# Submission Goals

- Goal: Successfully implement a a fully connected NN image classifier for CIFAR-10 with PyTorch Lightning

- Passing Criteria: Similar to the last homework, there are no unit tests that check specific components of your code. The only thing that's required to pass this optional submission, is your model to reach at least **50% accuracy** on the test dataset.

- Submission start: __Oct 9, 2022 20:00__
- Submission deadline : __Oct 22, 2022 15:59__ 
- You can make **$\infty$** submissions until the deadline. Your __best submission__ will be considered.